<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tool Calling Evals</h1>

The purpose of this notebook is:

- to evaluate the multi-step LLM logic involved in tool calling,
- to provide an experimental framework for users to iterate and improve on the default evaluation template.

## Install Dependencies and Import Libraries

In [1]:
%pip install -qq "arize-phoenix>=8.8.0" "arize-phoenix-otel>=0.8.0" llama-index-llms-openai openai gcsfs nest_asyncio langchain langchain-openai openinference-instrumentation-langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
import nest_asyncio
import pandas as pd
from phoenix.evals import (
    TOOL_CALLING_PROMPT_RAILS_MAP,
    TOOL_CALLING_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

nest_asyncio.apply()

# Generate App Usage Data

Let's begin by generating some data to use for our evaluation. Let's pretend we have a chatbot for an ecommerce company that is armed with a set of functions to lookup products and orders.

In [4]:
GEN_TEMPLATE = """
You are an assistant that generates complex customer service questions. You will try to answer the question with the tool if possible,
do your best to answer, ask for more information only if needed.
The questions should often involve:

Please reference the product names, the product details, product IDS and product information.

Multiple Categories: Questions that could logically fall into more than one category (e.g., combining product details with a discount code).
Vague Details: Questions with limited or vague information that require clarification to categorize correctly.
Mixed Intentions: Queries where the customer’s goal or need is unclear or seems to conflict within the question itself.
Indirect Language: Use of indirect or polite phrasing that obscures the direct need or request (e.g., using "I was wondering if..." or "Perhaps you could help me with...").
For specific categories:

Track Package: Include vague timing references (e.g., "recently" or "a while ago") instead of specific dates.
Product Comparison and Product Search: Include generic descriptors without specific product names or IDs (e.g., "high-end smartphones" or "energy-efficient appliances").
Apply Discount Code: Include questions about discounts that might apply to hypothetical or past situations, or without mentioning if they have made a purchase.
Product Details: Ask for comparisons or details that involve multiple products or categories ambiguously (e.g., "Tell me about your range of electronics that are good for home office setups").
Examples of More Challenging Questions
Multiple Categories

"I recently bought a samsung 106i smart phone, and I was wondering if there's a way to check what deals I might have missed or if my order is on its way?"
"Could you tell me if the samsung 15H adapater in my last order are covered under warranty and if they have shipped yet?"
Vague Details

"There's an issue with one of the Vizio 14Y TV I think I bought last month—what should I do?"
"I need help with a iPhone 16H I ordered, or maybe I'm just looking for something new. Can you help?"
Mixed Intentions

"I'm not sure if I should ask for a refund or just find out when it will arrive. What do you suggest?"
"Could you help me decide whether to upgrade my product or just track the current one?"
Indirect Language

"I was wondering if you might assist me in figuring out a problem I have with an order, or maybe it's more of a query?"
"Perhaps you could help me understand the benefits of your premium products compared to the regular ones?"

Some questions should be straightforward uses of the provided functions

Respond with a list, one question per line. Do not include any numbering at the beginning of each line. Do not include any category headings.
Generate 20 questions.
"""

In [5]:
model = OpenAIModel(model="gpt-4o", max_tokens=1300)

In [6]:
resp = model(GEN_TEMPLATE)

In [7]:
split_response = resp.strip().split("\n")

questions_df = pd.DataFrame(split_response, columns=["questions"])
print(questions_df)

                                            questions
0   I'm trying to find out if the Samsung 106i sma...
1   Could you help me understand the differences b...
2   I was wondering if there's a way to apply a di...
3   I need assistance with a product I ordered a w...
4   Could you tell me if there are any current dea...
5   I'm not sure if I should be looking for a refu...
6   Perhaps you could help me figure out if the pr...
7   I recently bought a product, but I'm not sure ...
8   Could you assist me in finding out if my order...
9   I was wondering if you might have any informat...
10  Could you help me decide whether to upgrade my...
11  I'm looking for a new smartphone, but I'm not ...
12  I need to know if there's a way to apply a dis...
13  Could you tell me about your range of electron...
14  I was wondering if you could assist me in unde...
15  I'm not sure if I should ask for a refund on m...
16  Could you help me with a problem I have with a...
17  I need help with a produ

# Define a tool-calling agent with Langchain

Now we'll define the chatbot agent using Langchain to attach the functions as tools.

In [8]:
import phoenix as px
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from phoenix.otel import register

### Connect to Phoenix

We'll also enable tracing with Phoenix using our Langchain auto-instrumentor to capture telemetry that we can later evaluate.

This code will connect you to an online version of Phoenix, at app.phoenix.arize.com. If you're self-hosting Phoenix, be sure to change your Collector Endpoint below, and remove the API Key.

In [9]:
if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = phoenix_api_key
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_api_key}"

os.environ["PHOENIX_PROJECT_NAME"] = "Tool Calling Eval"

tracer_provider = register(auto_instrument=True, project_name="Tool Calling Eval")

🔑 Enter your Phoenix API key:  ········


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Tool Calling Eval
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Now we'll define our basic functions using pydantic. The actual logic of the functions doesn't matter in this evaluation scenario, since we won't be evaluating anything beyond the function generation step.

In [10]:
## function definitions using pydantic decorator


@tool
def product_comparison(product_a_id: str, product_b_id: str) -> dict:
    """
    Compare features of two products.

    Parameters:
    product_a_id (str): The unique identifier of Product A.
    product_b_id (str): The unique identifier of Product B.

    Returns:
    dict: A dictionary containing the comparison of the two products.
    """

    if product_a_id == "" or product_b_id == "":
        return {"error": "missing product id"}

    # Implement the function logic here
    return {"comparison": "Similar"}


@tool
def product_details(product_id: str) -> dict:
    """
    Get detailed features on one product.

    Parameters:
    product_id (str): The unique identifier of the Product.

    Returns:
    dict: A dictionary containing product details.
    """

    if product_id == "":
        return {"error": "missing product id"}

    # Implement the function logic here
    return {"name": "Product Name", "price": "$12.50", "Availability": "In Stock"}


@tool
def apply_discount_code(order_id: int, discount_code: str) -> dict:
    """
    Applies a discount code to an order.

    Parameters:
    order_id (str): The unique identifier of the order.
    discount_code (str): The discount code to apply.

    Returns:
    dict: A dictionary containing the updated order details.
    """

    if order_id == "" or discount_code == "":
        return {"error": "missing order id or discount code"}

    # Implement the function logic here
    return {"applied": "True"}


@tool
def product_search(
    query: str,
    category: str = None,
    min_price: float = 0.0,
    max_price: float = None,
    page: int = 1,
    page_size: int = 20,
) -> dict:
    """
    Search for products based on criteria.

    Parameters:
    query (str): The search query string.
    category (str, optional): The category to filter the search. Default is None.
    min_price (float, optional): The minimum price of the products to search. Default is 0.
    max_price (float, optional): The maximum price of the products to search. Default is None.
    page (int, optional): The page number for pagination. Default is 1.
    page_size (int, optional): The number of results per page. Default is 20.

    Returns:
    dict: A dictionary containing the search results and pagination info.
    """

    if query == "":
        return {"error": "missing query"}

    # Implement the function logic here
    return {"results": [], "pagination": {"total": 0, "page": 1, "page_size": 20}}


@tool
def customer_support(issue_type: str) -> dict:
    """
    Get contact information for customer support regarding an issue.

    Parameters:
    issue_type (str): The type of issue (e.g., billing, technical support).

    Returns:
    dict: A dictionary containing the contact information for customer support.
    """

    if issue_type == "":
        return {"error": "missing issue type"}

    # Implement the function logic here
    return {"contact": issue_type}


@tool
def track_package(tracking_number: int) -> dict:
    """
    Track the status of a package based on the tracking number.

    Parameters:
    tracking_number (str): The tracking number of the package.

    Returns:
    dict: A dictionary containing the tracking status of the package.
    """
    if tracking_number == "":
        return {"error": "missing tracking number"}

    # Implement the function logic here
    return {"status": "Delivered"}


tools = [
    product_comparison,
    product_search,
    customer_support,
    track_package,
    apply_discount_code,
    product_details,
]

In [11]:
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/Users/thongbuiwork/.pyenv/versions/3.11.4/envs/phoenix-eval/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Run Chain on each question

With our agent defined, we can now run it across each generated question.

In [12]:
questions_df["response"] = questions_df["questions"].apply(
    lambda x: agent_executor.invoke({"input": x})
)

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 




> Entering new AgentExecutor chain...


Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 



Invoking: `product_details` with `{'product_id': 'Samsung 106i'}`


{'name': 'Product Name', 'price': '$12.50', 'Availability': 'In Stock'}
Invoking: `track_package` with `{'tracking_number': 123456789}`


{'status': 'Delivered'}

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


...


Invoking: `product_search` with `{'query': 'high-end smartphone', 'page_size': 5}`


{'results': [], 'pagination': {'total': 0, 'page': 1, 'page_size': 20}}

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


It seems that there are currently no high-end smartphone deals listed at the moment. If you have a specific model or brand in mind, I can try searching or comparing those for you. Let me know if there's anything else specific you're looking for!

> Finished chain.


> Entering new AgentExecutor chain...


Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


Of course, I can help with that. Here are a few steps we can take to assist in your decision:

1. **Upgrade Consideration**: If you're thinking about upgrading, we can compare the features of your current product with a potential upgrade to see if it's worth it.

2. **Track Shipping Status**: If you'd like to know where your current product is and when it will arrive, I can help track its shipping status.

Please let me know the product ID of your current item and, if applicable, the product ID of the potential upgrade you are considering. Additionally, provide your tracking number if you'd like to check the shipping status.

> Finished chain.


In [13]:
questions_df

,questions,response
0,I'm trying to find out if the Samsung 106i sma...,{'input': 'I'm trying to find out if the Samsu...
1,Could you help me understand the differences b...,{'input': 'Could you help me understand the di...
2,I was wondering if there's a way to apply a di...,{'input': 'I was wondering if there's a way to...
3,I need assistance with a product I ordered a w...,{'input': 'I need assistance with a product I ...
4,Could you tell me if there are any current dea...,{'input': 'Could you tell me if there are any ...
5,I'm not sure if I should be looking for a refu...,{'input': 'I'm not sure if I should be looking...
6,Perhaps you could help me figure out if the pr...,{'input': 'Perhaps you could help me figure ou...
7,"I recently bought a product, but I'm not sure ...","{'input': 'I recently bought a product, but I'..."
8,Could you assist me in finding out if my order...,{'input': 'Could you assist me in finding out ...
9,I was wondering if you might have any informat...,{'input': 'I was wondering if you might have a...


# Evaluate Tool Calls

Now that we have some example runs of our agent to analyze, we can start the evaluation process. We'll start by exporting all of those spans from Phoenix

In [14]:
from phoenix.trace import SpanEvaluations
from phoenix.trace.dsl import SpanQuery

Since we'll only be evaluating the inputs, outputs, and function call columns, let's extract those into an easier to use df. Helpfully, Phoenix provides a method to query your span data and directly export only the values you care about.

In [15]:
query = (
    SpanQuery()
    .where(
        # Filter for the `LLM` span kind.
        # The filter condition is a string of valid Python boolean expression.
        "span_kind == 'LLM'",
    )
    .select(
        # Extract and rename the following span attributes
        question="llm.input_messages",
        outputs="llm.output_messages",
    )
)
trace_df = px.Client().query_spans(query, project_name="Tool Calling Eval")
# trace_df["tool_call"] = trace_df["tool_call"].fillna("No tool used")

/Users/thongbuiwork/.pyenv/versions/3.11.4/envs/phoenix-eval/lib/python3.11/site-packages/phoenix/utilities/client.py:40: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


HTTPStatusError: Client error '401 Unauthorized' for url 'https://app.phoenix.arize.com/v1/spans?project_name=Tool+Calling+Eval&project-name=Tool+Calling+Eval'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

In [ ]:
def get_tool_call(outputs):
    if outputs[0].get("message").get("tool_calls"):
        return (
            outputs[0]
            .get("message")
            .get("tool_calls")[0]
            .get("tool_call")
            .get("function")
            .get("name")
        )
    else:
        return "No tool used"


trace_df["tool_call"] = trace_df["outputs"].apply(get_tool_call)

We'll also need to pass in our tool definitions to the evaluator:

In [ ]:
tool_definitions = ""

for current_tool in tools:
    tool_definitions += f"""
    {current_tool.name}: {current_tool.description}
    """

print(tool_definitions)

Next, we define the evaluator model to use

In [ ]:
trace_df["tool_definitions"] = tool_definitions

In [ ]:
eval_model = OpenAIModel(model="gpt-4o")

And we're ready to call our evaluator! The method below takes in the dataframe of traces to evaluate, our built in evaluation prompt, the eval model to use, and a rails object to snap responses from our model to a set of binary classification responses.

We'll also instruct our model to provide explanations for its responses.

In [ ]:
rails = list(TOOL_CALLING_PROMPT_RAILS_MAP.values())

response_classifications = llm_classify(
    dataframe=trace_df,
    template=TOOL_CALLING_PROMPT_TEMPLATE,
    model=eval_model,
    rails=rails,
    provide_explanation=True,
)
response_classifications["score"] = response_classifications.apply(
    lambda x: 1 if x["label"] == "correct" else 0, axis=1
)

In [ ]:
response_classifications

Finally, we'll export these responses back into Phoenix to view them in the UI.

In [ ]:
px.Client().log_evaluations(
    SpanEvaluations(eval_name="Tool Calling Eval", dataframe=response_classifications),
)

From here, you could iterate on different agent logic and prompts to improve performance, or you could further decompose the evaluation into individual steps looking first at Routing, then Parameter Extraction, then Code Generation to determine where to focus.

![Tool Calling Evaluation Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/tool-calling-nb-result.png)

Happy building!